#Exercício em Python - Tema 5 - Evapotranspiração e Demandas de Água

Neste exercício, você aprenderá a calcular a evapotranspiração Potencial (ETP) para diferentes biomas usando o método de Penman-Monteith, amplamente utilizado para estimar a ET com base em variáveis climáticas. A partir disso, será possível comparar as demandas de irrigação em diferentes cenários.

##Importar as bibliotecas necessárias e base de dados CABra

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io

##Importação dos dados

In [ ]:
#Importar dados gerais e de biomas de cada bacia
url = ("https://raw.githubusercontent.com/brunokenmarchezepe/CABra_exercicios/refs/heads/main/CABRA_dataset/CABra_attributes/CABra_general_attributes.txt")

bacia_df = requests.get(url).content
bacia_df = pd.read_csv(io.StringIO(bacia_df.decode('utf-8', errors = 'ignore')),
                 skiprows =8,
                 delimiter='\t')
bacia_df.columns = ['ID','ANA ID','longitude','latitude','gauge_hreg','gauge_biome','gauge_state','missing_data','series_length','quality_index']
bacia_df = bacia_df.drop(['ANA ID','gauge_hreg','missing_data', 'longitude', 'latitude', 'series_length','quality_index'], axis=1) #Excluindo colunas que não importam na análise

#Importar dados climáticos de cada bacia
url = ("https://raw.githubusercontent.com/brunokenmarchezepe/CABra_exercicios/refs/heads/main/CABRA_dataset/CABra_attributes/CABra_climate_attributes.txt")

bacia_clim = requests.get(url).content
bacia_clim = pd.read_csv(io.StringIO(bacia_clim.decode('utf-8', errors = 'ignore')),
                 skiprows =8,
                 delimiter='\t')
bacia_clim.columns = ['ID','ANA ID','clim_p','clim_tmin','clim_tmax','clim_rh','clim_wind','clim_srad','clim_et','clim_pet','aridity_index','p_seasonality','clim_quality']
bacia_clim = bacia_clim.drop(['ANA ID', 'clim_quality'], axis=1)
bacia_df = pd.merge(bacia_df, bacia_clim, on = ['ID'])

bacia_df.head()

,ID,gauge_biome,gauge_state,clim_p,clim_tmin,clim_tmax,clim_rh,clim_wind,clim_srad,clim_et,clim_pet,aridity_index,p_seasonality
0,1,Amazon,AM,5.64,21.61,30.48,83.17,0.82,16.42,3.87,5.04,0.89,0.46
1,2,Amazon,AM,8.88,21.83,28.60,86.41,0.42,16.24,4.29,4.96,0.56,-0.28
2,3,Amazon,AM,8.52,22.18,29.67,85.71,0.60,15.60,4.28,4.81,0.56,-0.27
3,4,Amazon,AM,8.86,22.00,29.13,86.26,0.49,15.99,4.29,4.90,0.55,-0.26
4,5,Amazon,AM,7.79,21.91,28.80,85.44,0.67,16.14,4.21,4.94,0.63,-0.27


##Definir a função para calcular a Evapotranspiração Potencial

In [ ]:
def penman_monteith(T, RH, u2, Rs):
    # Constantes
    G = 0  # fluxo de calor no solo, assumido como zero
    gamma = 0.065  # coeficiente psicrométrico, kPa/°C
    delta = 4098 * (0.6108 * np.exp((17.27 * T) / (T + 237.3))) / ((T + 237.3)**2)  # declividade da curva de pressão de vapor
    Rn = 0.77 * Rs  # radiação líquida, assumindo albedo de 0.23
    es = 0.6108 * np.exp((17.27 * T) / (T + 237.3))  # pressão de vapor de saturação
    ea = es * (RH / 100)  # pressão de vapor real

    # Fórmula de Penman-Monteith
    ETP = (0.408 * delta * (Rn - G) + gamma * (900 / (T + 273)) * u2 * (es - ea)) / (delta + gamma * (1 + 0.34 * u2))
    return ETP

##Calcular a ETP

In [ ]:
# Aplica a função Penman-Monteith para cada linha do dataframe
bacia_df['ETP_calculated'] = bacia_df.apply(lambda row: penman_monteith(row['clim_tmax'], row['clim_rh'], row['clim_wind'], row['clim_srad']), axis=1)

# Exibe as primeiras linhas com a coluna de ETP calculada
print(bacia_df[['ID', 'clim_tmax', 'clim_rh', 'clim_wind', 'clim_srad', 'ETP_calculated']].head())

In [ ]:
#Criação de uma seleção das bacias de ID 1, 53 e 130
selec = bacia_df.loc[bacia_df['ID'].isin([1, 53, 130])].copy()

#Cálculo da temperatura média
selec['clim_tmean'] = (selec['clim_tmax'] + selec['clim_tmin']) / 2

# Cálculo da ETP para cada bacia
selec['ETP'] = selec.apply(lambda row: penman_monteith(row['clim_tmax'], row['clim_rh'], row['clim_wind'], row['clim_srad']), axis=1)

print(selec[['gauge_biome', 'ETP']])

               gauge_biome       ETP
0                   Amazon  4.217495
52                 Cerrado  5.037602
129               Caatinga  6.234804


##Estimar as demandas de irrigação

In [ ]:
# Coeficiente de cultivo (Kc) para diferentes culturas
culturas = {'Milho': 1.2, 'Soja': 1.15, 'Cana': 1.1}

# Estimativa de demanda de irrigação para cada bioma e cultura
for cultura, Kc in culturas.items():
    selec[f'Demanda {cultura}'] = selec['ETP'] * Kc

print(selec[['gauge_biome', 'Demanda Milho', 'Demanda Soja', 'Demanda Cana']])

               gauge_biome  Demanda Milho  Demanda Soja  Demanda Cana
0                   Amazon       5.060994      4.850119      4.639244
52                 Cerrado       6.045123      5.793243      5.541362
129               Caatinga       7.481765      7.170025      6.858284
